# 期货实盘交易大赛数据分析

数据来源:
- 牛钱网 (niumoney.com)
- 七禾网 (trader.7hcn.com)
- 期货日报 (spds.qhrb.com.cn)

In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import plotly.io as pio
pio.renderers.default = 'notebook'

from scrapers import NiumoneyScraper, Qihuo7hcnScraper
from storage import StorageManager
from analysis import RankingAnalyzer, PerformanceAnalyzer, TrendAnalyzer
from visualization import CompetitionCharts

## 1. 数据采集

In [ ]:
# 牛钱网 - 2026年度排行榜
nm = NiumoneyScraper()
nm_rows = nm.scrape_ranking(tid=143, max_pages=3)
print(f'牛钱网: {len(nm_rows)} 条记录')

# 七禾网 - 2025-2026颁奖年度排行
qh = Qihuo7hcnScraper()
qh_rows = qh.scrape_ranking(tid='2025-2026', tag='nav', max_pages=3)
print(f'七禾网: {len(qh_rows)} 条记录')

In [ ]:
# 保存数据
storage = StorageManager()
storage.save_rows(nm_rows, 'niumoney', 'year_2026')
storage.save_rows(qh_rows, 'qihuo7hcn', '2025-2026')

In [ ]:
df_nm = pd.DataFrame(nm_rows)
df_qh = pd.DataFrame(qh_rows)
print('牛钱网数据字段:', df_nm.columns.tolist())
print('七禾网数据字段:', df_qh.columns.tolist())
df_nm.head()

## 2. 排行榜统计分析

In [ ]:
ra = RankingAnalyzer(df_nm)

print('=== 总体概况 ===')
summary = ra.summary()
for k, v in summary.items():
    print(f'  {k}: {v}')

print('\n=== 收益率分布 ===')
dist = ra.return_distribution()
for k, v in dist.items():
    print(f'  {k}: {v}')

print('\n=== 盈亏分析 ===')
pl = ra.profit_loss_analysis()
for k, v in pl.items():
    print(f'  {k}: {v}')

In [ ]:
# 期货公司统计
company = ra.company_stats()
company.head(15)

In [ ]:
# Top 20 选手
ra.top_n(20, by='profit_rate')

## 3. 选手表现分析

In [ ]:
pa = PerformanceAnalyzer(df_qh)

print('=== 收益统计 ===')
stats = pa.return_statistics()
for k, v in stats.items():
    if isinstance(v, dict):
        for kk, vv in v.items():
            print(f'  {kk}: {vv:.2f}')
    else:
        print(f'  {k}: {v}')

print('\n=== 回撤分析 ===')
dd = pa.drawdown_analysis()
for k, v in dd.items():
    print(f'  {k}: {v}')

In [ ]:
# 风险收益概况
profile = pa.risk_return_profile()
profile.head(10)

In [ ]:
# 指标相关性
corr = pa.correlation_matrix()
corr

## 4. 数据可视化

In [ ]:
charts = CompetitionCharts()

# 收益率分布
fig = charts.return_distribution(df_nm, title='牛钱网 2026年度 收益率分布')
fig.show()

In [ ]:
# 风险收益散点图
fig = charts.risk_return_scatter(df_nm, title='牛钱网 风险收益散点图')
fig.show()

In [ ]:
# 资金规模分布
fig = charts.equity_pie(df_nm, title='牛钱网 资金规模分布')
fig.show()

In [ ]:
# 期货公司参赛统计
fig = charts.company_bar(company, title='牛钱网 期货公司参赛统计 Top 20')
fig.show()

In [ ]:
# 综合仪表盘
fig = charts.dashboard(df_nm, title='牛钱网 2026年度 综合分析仪表盘')
fig.show()

## 5. 品种盈利排行（七禾网）

In [ ]:
# 获取多个品种的排行数据
varieties = ['AU', 'CU', 'RB', 'SC', 'M']
variety_data = {}
for code in varieties:
    rows = qh.scrape_futures_ranking(code, max_pages=1)
    variety_data[code] = pd.DataFrame(rows)
    print(f'{code}: {len(rows)} 条记录')

# 品种盈利排名对比
for code, vdf in variety_data.items():
    if not vdf.empty:
        print(f'\n--- {code} Top 5 ---')
        print(vdf[['rank', 'nickname', 'profit', 'win_rate']].head())

## 6. 趋势追踪（跨周期对比）

In [ ]:
# 采集多个月度数据用于趋势分析
from scrapers.niumoney import RANKING_TIDS

ta = TrendAnalyzer()
month_tids = {
    '2026_01': RANKING_TIDS.get('month_2026_01', 141),
    '2026_02': RANKING_TIDS.get('month_2026_02', 142),
}

for label, tid in month_tids.items():
    rows = nm.scrape_ranking(tid=tid, max_pages=2)
    if rows:
        ta.add_snapshot(label, pd.DataFrame(rows))
        print(f'{label}: {len(rows)} 条')

# 市场概况
overview = ta.market_overview()
overview

In [ ]:
# 市场概况趋势图
if not overview.empty:
    fig = charts.market_overview(overview)
    fig.show()